In [ ]:
#!/usr/bin/env python3

import argparse
import json
import os
import sys
import joblib


class SpamDetector:

    def __init__(self, model_path='spam_classifier_final.pkl'):
        try:
            if not os.path.exists(model_path):
                print("Model not found. Starting model training...")
                self._train_model()
            else:
                self.artifact = joblib.load(model_path)
                self.model = self.artifact['production_model']
                self.label_encoder = self.artifact['label_encoder']
                self.optimal_threshold = self.artifact['optimal_threshold']
                self.metadata = self.artifact['metadata']
        except Exception as e:
            print(f"Error loading model: {e}")
            sys.exit(1)

    def _train_model(self):
        print("Authentication required for model training.")
        password = input("Enter admin password: ")

        if password != "qazwsxE78":
            print("Wrong password. Access denied.")
            sys.exit(1)

        print("Starting model training process...")
        from original_model_trainer import main as train_main
        train_main()

        self.artifact = joblib.load('spam_classifier_final.pkl')
        self.model = self.artifact['production_model']
        self.label_encoder = self.artifact['label_encoder']
        self.optimal_threshold = self.artifact['optimal_threshold']
        self.metadata = self.artifact['metadata']

    def detect_boolean(self, message):
        try:
            proba = self.model.predict_proba([message])[:, 1]
            return proba[0] >= self.optimal_threshold
        except Exception as e:
            print(f"Classification error: {e}")
            return False

    def detect_basic(self, message):
        try:
            is_spam = self.detect_boolean(message)

            if is_spam:
                result_text = "SPAM"
                explanation = "Message classified as spam"
            else:
                result_text = "HAM"
                explanation = "Message classified as legitimate"

            return {
                "result": result_text,
                "explanation": explanation,
                "recommendation": "Ignore message" if is_spam else "Message is safe"
            }

        except Exception as e:
            return {
                "result": "ERROR",
                "explanation": f"Analysis error: {e}",
                "recommendation": "Try again"
            }

    def detect_pro(self, message):
        try:
            is_spam = self.detect_boolean(message)
            proba = self.model.predict_proba([message])[:, 1][0]
            ham_prob = 1 - proba

            return {
                "spam_probability": round(proba, 4),
                "ham_probability": round(ham_prob, 4),
                "classification_threshold": round(self.optimal_threshold, 4),
                "final_verdict": "SPAM" if is_spam else "HAM",
                "model_performance": {
                    "average_precision": self.metadata['performance']['average_precision'],
                    "f1_score": self.metadata['performance']['f1_score_optimized']
                }
            }

        except Exception as e:
            return {
                "error": f"Analysis error: {e}",
                "final_verdict": "ERROR"
            }


def main():
    parser = argparse.ArgumentParser(description='SMS Spam Detection Utility')
    parser.add_argument(
        '--mode',
        type=str,
        required=True,
        choices=['boolean', 'basic', 'pro'],
        help='Operation mode: boolean, basic, pro'
    )
    parser.add_argument(
        '--message',
        type=str,
        required=True,
        help='Message to analyze'
    )
    parser.add_argument(
        '--model',
        type=str,
        default='spam_classifier_final.pkl',
        help='Path to model file'
    )

    args = parser.parse_args()

    detector = SpamDetector(args.model)

    if args.mode == 'boolean':
        result = detector.detect_boolean(args.message)
        print(json.dumps({"is_spam": result}, ensure_ascii=False))

    elif args.mode == 'basic':
        result = detector.detect_basic(args.message)
        print(json.dumps(result, ensure_ascii=False, indent=2))

    elif args.mode == 'pro':
        result = detector.detect_pro(args.message)
        print(json.dumps(result, ensure_ascii=False, indent=2))


if __name__ == "__main__":
    main()